# 실습 2. 서울시 상권별 월평균 매출액 예측  
1. 데이터: 서울시 우리마을 가게 상권배후지추정매출 (2014년 ~ 2020년 2분기)
2. 예측: 상권 종목 별 월평균 매출액 예측 (KRW)
3. 특징: 시계열 데이터 변환
4. 목표: 분기별 예측 R2 80% 이상

### Hint  
pivot_table()  
데이터 전처리를 할 때 상권의 매출들을 분기별로 컬럼을 만듬  
매출 / 점포 수 = 점포 당 평균 매출  
매출이 0인 부분은 영업하지 않을 때  
시각화를 한다면 연령별 데이터, 시간대별 데이터를 사용  
but, 이번 문제에서는 매출에 집중하기에 time series를 함  
가장 최근의 값인, 2020-2분기의 월매출을 y로, 나머지 데이터는 X

### 데이터 불러오기

### 생략없이 지정된 범위를 한번에 보도록
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

In [1]:
import pandas as pd
import numpy as np
import os
forders = os.listdir('../data/상권배후지추정매출')
df_all_years = pd.DataFrame()
for i in range(0,len(forders)):
    path = '../data/상권배후지추정매출/'+forders[i]
    df= pd.read_csv(path,encoding='cp949',thousands = ',') 
    df_all_years = pd.concat([df_all_years, df])
df_all_years.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2014,1,A,골목상권,1000522,목동로9길,CS200041,사진관,161604397.0,3021,...,63,1450,1435,97,683,679,674,441,311,5
1,2014,1,A,골목상권,1000580,공항대로8길,CS100007,치킨전문점,352909128.0,14422,...,5746,8280,5597,256,3970,4688,2750,1707,510,19
2,2014,1,A,골목상권,1000507,가로공원로58길,CS200028,미용실,174625276.0,5695,...,289,2639,3044,130,1234,1561,1440,925,391,36
3,2014,1,A,골목상권,1000556,강서로18길,CS300035,인테리어,158530192.0,1571,...,30,395,1158,20,127,662,366,286,92,21
4,2014,1,A,골목상권,1000509,곰달래로2길,CS100007,치킨전문점,238151311.0,9718,...,4157,5763,3573,171,2165,3484,2160,1038,316,11


### 데이터 확인

In [2]:
df_all_years.shape

(1434810, 80)

In [3]:
df_all_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1434810 entries, 0 to 96182
Data columns (total 80 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   기준_년_코드          1434810 non-null  int64  
 1   기준_분기_코드         1434810 non-null  int64  
 2   상권_구분_코드         1434810 non-null  object 
 3   상권_구분_코드_명       1434810 non-null  object 
 4   상권_코드            1434810 non-null  int64  
 5   상권_코드_명          1434810 non-null  object 
 6   서비스_업종_코드        1434810 non-null  object 
 7   서비스_업종_코드_명      1434810 non-null  object 
 8   당월_매출_금액         1434810 non-null  float64
 9   당월_매출_건수         1434810 non-null  int64  
 10  주중_매출_비율         1434810 non-null  int64  
 11  주말_매출_비율         1434810 non-null  int64  
 12  월요일_매출_비율        1434810 non-null  int64  
 13  화요일_매출_비율        1434810 non-null  int64  
 14  수요일_매출_비율        1434810 non-null  int64  
 15  목요일_매출_비율        1434810 non-null  int64  
 16  금요일_매출_비율        143

In [4]:
df_all_years.isna().sum()

기준_년_코드            0
기준_분기_코드           0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
                  ..
연령대_30_매출_건수       0
연령대_40_매출_건수       0
연령대_50_매출_건수       0
연령대_60_이상_매출_건수    0
점포수                0
Length: 80, dtype: int64

In [5]:
df_all_years['서비스_업종_코드'].nunique()

82

### 점포수가 0인 컬럼값을 1로 변경

In [6]:
df_all_years[(df_all_years['점포수'] == 0)]['점포수'].value_counts()

0    8657
Name: 점포수, dtype: int64

In [7]:
df_all_years[(df_all_years['점포수'] == 0)] = 1
df_all_years[(df_all_years['점포수'] == 0)]['점포수'].value_counts()

Series([], Name: 점포수, dtype: int64)

### 기준년도 + 분기 컬럼 추가

In [8]:
df_all_years["년도_분기"] = df_all_years['기준_년_코드'].astype(str) +"_" +df_all_years['기준_분기_코드'].astype(str)
df_all_years["년도_분기"]

0        2014_1
1        2014_1
2        2014_1
3        2014_1
4        2014_1
          ...  
96178    2020_1
96179    2020_1
96180    2020_1
96181    2020_1
96182    2020_1
Name: 년도_분기, Length: 1434810, dtype: object

### 점포당 평균 매출 컬럼 추가

In [9]:
df_all_years["점포당_평균_월매출"] = df_all_years['당월_매출_금액'] / df_all_years['점포수']
df_all_years["점포당_평균_월매출"].head(5)

0    3.232088e+07
1    1.857416e+07
2    4.850702e+06
3    7.549057e+06
4    2.165012e+07
Name: 점포당_평균_월매출, dtype: float64

### pivot_table 사용

In [10]:
df_all_years["서비스_업종_코드"] = pd.to_numeric(df_all_years['서비스_업종_코드'].str.replace('CS',''))

In [11]:
df = df_all_years.pivot_table(index = ["상권_코드","서비스_업종_코드"], 
                        columns = "년도_분기", 
                        values = "점포당_평균_월매출")

### pivot_table 확인

In [12]:
df.head(10)

년도_분기                    2014_1        2014_2        2014_3        2014_4  \
상권_코드   서비스_업종_코드                                                           
1000001 100001.0   6.092752e+07  7.119277e+07  6.941415e+07  8.371927e+07   
        100002.0   7.830883e+07  7.419328e+07  7.638027e+07  9.373318e+07   
        100003.0   7.725436e+07  2.934658e+07  4.053009e+07  3.842372e+07   
        100004.0   8.113118e+07  9.185987e+07  9.243012e+07  1.814898e+08   
        100005.0   3.476190e+07  3.927323e+07  2.214167e+07  2.165559e+07   
        100006.0   3.838813e+07  4.349731e+07  3.736621e+07  4.097061e+07   
        100007.0   2.653956e+07  4.299558e+07  3.992420e+07  4.632464e+07   
        100008.0   7.276838e+07  7.715464e+07  6.714847e+07  7.734539e+07   
        100009.0   6.285681e+07  6.776887e+07  5.827576e+07  5.458187e+07   
        100010.0   4.857559e+07  5.807126e+07  6.404752e+07  5.528082e+07   

년도_분기                    2015_1        2015_2        2015_3        2015_4  \
상권_코드   서비스_업종_코드                                                           
1000001 100001.0   6.258971e+07  6.645696e+07  6.576536e+07  7.757226e+07   
        100002.0   6.686465e+07  7.288815e+07  7.048284e+07  8.242502e+07   
        100003.0   2.944173e+07  3.137040e+07  3.018033e+07  3.600755e+07   
        100004.0   9.864977e+07  1.076278e+08  9.826663e+07  1.204825e+08   
        100005.0   6.659322e+07  6.389587e+07  4.910766e+07  4.643170e+07   
        100006.0   3.176743e+07  3.248449e+07  2.595889e+07  4.025296e+07   
        100007.0   2.298493e+07  2.549281e+07  3.079232e+07  2.987025e+07   
        100008.0   7.565224e+07  7.484982e+07  6.785900e+07  8.002915e+07   
        100009.0   5.904118e+07  6.178512e+07  6.015102e+07  7.463980e+07   
        100010.0   5.676480e+07  6.637749e+07  6.264861e+07  5.094684e+07   

년도_분기                    2016_1        2016_2  ...        2018_1  \
상권_코드   서비스_업종_코드                              ...                 
1000001 100001.0   7.015885e+07  7.694477e+07  ...  7.142265e+07   
        100002.0   7.914830e+07  8.162432e+07  ...  6.538066e+07   
        100003.0   2.957095e+07  2.134013e+07  ...  1.566458e+07   
        100004.0   1.249218e+08  1.199577e+08  ...  9.912141e+07   
        100005.0   6.138270e+07  5.458803e+07  ...  4.145602e+07   
        100006.0   4.702610e+07  3.121249e+07  ...  6.226979e+07   
        100007.0   2.390353e+07  2.880941e+07  ...  2.457452e+07   
        100008.0   5.320424e+07  5.981367e+07  ...  4.916738e+07   
        100009.0   5.256283e+07  7.108628e+07  ...  1.139984e+08   
        100010.0   4.561565e+07  5.928037e+07  ...  5.399234e+07   

년도_분기                    2018_2        2018_3        2018_4        2019_1  \
상권_코드   서비스_업종_코드                                                           
1000001 100001.0   8.301902e+07  7.379520e+07  8.717491e+07  6.061903e+07   
        100002.0   7.679618e+07  5.656538e+07  8.571366e+07  6.102188e+07   
        100003.0   1.598102e+07  2.662950e+07  2.784196e+07  2.361333e+07   
        100004.0   1.228614e+08  1.113682e+08  1.434638e+08  8.218444e+07   
        100005.0   3.998182e+07  3.790789e+07  4.543281e+07  3.101623e+07   
        100006.0   8.700248e+07  6.731598e+07  7.111364e+07  5.972577e+07   
        100007.0   2.943043e+07  2.951593e+07  2.752145e+07  3.031635e+07   
        100008.0   5.800322e+07  4.850796e+07  5.975140e+07  6.377430e+07   
        100009.0   1.907070e+08  1.971140e+08  2.160490e+08  4.665696e+07   
        100010.0   7.485417e+07  6.161511e+07  8.432246e+07  4.584765e+07   

년도_분기                    2019_2        2019_3        2019_4        2020_1  \
상권_코드   서비스_업종_코드                                                           
1000001 100001.0   6.990543e+07  6.424871e+07  7.248866e+07  4.478399e+07   
        100002.0   6.731579e+07  6.169022e+07  6.127224e+07  4.493084e+07   
        100003.0   4.750040e+06  5.968620e+07  5.298060e+07  3.127229e+07   
        100004.0   7.41

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 63428 entries, (1000001, 100001.0) to (1001010, 300043.0)
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2014_1  56023 non-null  float64
 1   2014_2  56409 non-null  float64
 2   2014_3  56576 non-null  float64
 3   2014_4  56614 non-null  float64
 4   2015_1  56433 non-null  float64
 5   2015_2  56798 non-null  float64
 6   2015_3  56769 non-null  float64
 7   2015_4  56825 non-null  float64
 8   2016_1  56596 non-null  float64
 9   2016_2  57068 non-null  float64
 10  2016_3  57355 non-null  float64
 11  2016_4  57476 non-null  float64
 12  2017_1  57204 non-null  float64
 13  2017_2  57338 non-null  float64
 14  2017_3  56972 non-null  float64
 15  2017_4  56675 non-null  float64
 16  2018_1  56585 non-null  float64
 17  2018_2  56699 non-null  float64
 18  2018_3  56385 non-null  float64
 19  2018_4  56397 non-null  float64
 20  2019_1  49038 non-null  float64
 21  201

### 결측치 제거

In [14]:
df.dropna(axis=0, inplace = True)
df.isna().sum()

년도_분기
2014_1    0
2014_2    0
2014_3    0
2014_4    0
2015_1    0
2015_2    0
2015_3    0
2015_4    0
2016_1    0
2016_2    0
2016_3    0
2016_4    0
2017_1    0
2017_2    0
2017_3    0
2017_4    0
2018_1    0
2018_2    0
2018_3    0
2018_4    0
2019_1    0
2019_2    0
2019_3    0
2019_4    0
2020_1    0
2020_2    0
dtype: int64

### Splitting Train/Test set

In [15]:
X = df.drop('2020_2',axis=1)
y = df['2020_2']

In [16]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=1004)
print("Shape of X_train: {}".format(X_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_test: {}".format(X_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (29946, 25)
Shape of y_train: (29946,)
Shape of X_test: (12835, 25)
Shape of y_test: (12835,)


### XGBRegressor

In [17]:
from sklearn import metrics
from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
y_pred = xgb.predict(X_test)

In [19]:
rmse = np.sqrt(metrics.mean_squared_error(y_test,y_pred))
mae = metrics.mean_absolute_error(y_test,y_pred)
r2  = metrics.r2_score(y_test,y_pred)
print("RMSE: %.4f, MAE: %.4f, R-squared:%.4f"%(rmse,mae,r2))

RMSE: 41137385.8102, MAE: 10842854.6884, R-squared:0.8484
